# 특정챔프, 특정판수만 조사


## 모듈 

In [1]:
#특정챔프, 특정판수만 조사



#이번시즌 전경기 전수조사

import requests
from pandas.io.json import json_normalize
import requests
import pandas as pd
import numpy as np
import time
from pandas.io.json import json_normalize
import requests
import pandas as pd
import numpy as np
import time
import pandas as pd
import pickle

import pickle # 리스트 안의 데이터프레임 형태 저장
import requests # api 요청
import json
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import time

from skimage import io # 미니맵 처리
from sklearn.preprocessing import MinMaxScaler
#Self Created Modules
#############################################################################
from champ_key_trans import champ_key_trans as ckt

#############################################################################

## 검색 파라미터- 소환사이름, api키, 검색할 챔프, 검색할 판수

In [14]:
summonerName='빠른 쓰레기'
api_key='RGAPI-04456db4-d63b-43a6-bf9b-36f95f6357ed'
api_key2='RGAPI-9116dbba-34ed-45aa-8904-f2b2acb2b3c6'
api_key3='RGAPI-549999b4-9c33-4cd2-beb4-f46ce5091c7c'
Champion="Quinn"
Count=210

## 코드

In [11]:





def get_accountId(summonerName, api_key):
    summonerId='https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+summonerName+ '?api_key=' + api_key
    R_summonerId = requests.get(summonerId)
    accountId=R_summonerId.json()['accountId']
    
    return accountId

def get_summonerId(summonerName, api_key):
    summonerId='https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+summonerName+ '?api_key=' + api_key
    R_summonerId = requests.get(summonerId)
    summonerId=R_summonerId.json()['id']
   
    return summonerId

def get_TotalGamesCount(summonerId, api_key):
    TGC_link='https://kr.api.riotgames.com/lol/league/v4/entries/by-summoner/' + summonerId+ '?api_key='+api_key
    R_TGC_link=requests.get(TGC_link)

    TotalGamesCount=R_TGC_link.json()[0]['wins']+R_TGC_link.json()[0]['losses']

    return TotalGamesCount

def get_matchlist(accountId, championName, Gamecount, api_key):
    matchlist=[]
    championkey=ckt(championName)
    
    
    if Gamecount < 100:
        endIndex=Gamecount
        beginIndex=0


        matchlistlink=('https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/'+accountId+'?champion='+str(championkey)+'&queue=420&endIndex='+str(endIndex)+'&beginIndex='+str(beginIndex)+'&api_key='+api_key)
        #matchlistlink='https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + accountId+ '?queue=420&endIndex='+ str(endIndex)+'&beginIndex='+str(beginIndex)+'&api_key='+api_key
        R_matchList=requests.get(matchlistlink)
        for a in (R_matchList.json()['matches']):
            matchlist.append(a['gameId'])
    else:
    
        for hundred in range(int(Gamecount/100)):
            endIndex=(hundred+1)*100
            beginIndex=hundred*100


            matchlistlink=('https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/'+accountId+'?champion='+str(championkey)+'&queue=420&endIndex='+str(endIndex)+'&beginIndex='+str(beginIndex)+'&api_key='+api_key)
            #matchlistlink='https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + accountId+ '?queue=420&endIndex='+ str(endIndex)+'&beginIndex='+str(beginIndex)+'&api_key='+api_key
            R_matchList=requests.get(matchlistlink)
            for a in (R_matchList.json()['matches']):
                matchlist.append(a['gameId'])

                        
        endIndex=Gamecount
        beginIndex=(hundred+1)*100


        matchlistlink=('https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/'+accountId+'?champion='+str(championkey)+'&queue=420&endIndex='+str(endIndex)+'&beginIndex='+str(beginIndex)+'&api_key='+api_key)
        #matchlistlink='https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + accountId+ '?queue=420&endIndex='+ str(endIndex)+'&beginIndex='+str(beginIndex)+'&api_key='+api_key
        R_matchList=requests.get(matchlistlink)
        for a in (R_matchList.json()['matches']):
            matchlist.append(a['gameId'])

    
    return matchlist


def get_matchIdData(matchId, api_key,api_key2, api_key3, changer):
    if (changer==1):
        
        matchIddatalink=('https://kr.api.riotgames.com/lol/match/v4/matches/' + str(matchId) +'?api_key='+api_key)
    elif (changer==2):
        matchIddatalink=('https://kr.api.riotgames.com/lol/match/v4/matches/' + str(matchId) +'?api_key='+api_key2)
    else:
        matchIddatalink=('https://kr.api.riotgames.com/lol/match/v4/matches/' + str(matchId) +'?api_key='+api_key3)
    R_matchIddata=requests.get(matchIddatalink)
    print('changer : ', changer,'  matchIdData')
    while (R_matchIddata.status_code != 200):
        print("iddata status code = ", R_matchIddata.status_code, 'changer :' , changer)
        
        
        print('try 3 second wait time To 429error')
        time.sleep(3)
        R_matchIddata=requests.get(matchIddatalink)
        
 
    #print(matchId, " Iddata get")
    return R_matchIddata.json()

def get_matchtimeline(matchId, api_key,api_key2, api_key3, changer):
    if (changer==1):
        matchtimelinelink=('https://kr.api.riotgames.com/lol/match/v4/timelines/by-match/'+str(matchId)+'?api_key='+api_key)
    elif (changer==2):
        matchtimelinelink=('https://kr.api.riotgames.com/lol/match/v4/timelines/by-match/'+str(matchId)+'?api_key='+api_key2)
    else:
        matchtimelinelink=('https://kr.api.riotgames.com/lol/match/v4/timelines/by-match/'+str(matchId)+'?api_key='+api_key3)
    R_matchtimeline=requests.get(matchtimelinelink)
    print('changer : ', changer,  'matchTimeline')
    while (R_matchtimeline.status_code != 200)  :
        
        print("timeline_status_code = ",R_matchtimeline.status_code, 'changer : ', changer)
        print('try 3 second wait time To 429error')
        time.sleep(3)
        R_matchtimeline=requests.get(matchtimelinelink)
    #print(matchId, " timeline data get")
    return R_matchtimeline.json()



def get_gameid_iddata_timeline_SUM(matchlist, api_key, api_key2, api_key3):
    gameid_iddata_timeline_SUM = []

    for i in range(0,len(matchlist)):
        imsi=[]
        changer=i%3

        matchId=matchlist[i]
        matchIdData=get_matchIdData(matchId,api_key, api_key2,api_key3, changer)
        if matchIdData['gameDuration']>720:
         
            matchtimeline=get_matchtimeline(matchId,api_key, api_key2,api_key3, changer)
            imsi.append(matchId)
            imsi.append(matchIdData)
            imsi.append(matchtimeline)

            gameid_iddata_timeline_SUM.append(imsi)
            imsi=[]
    return gameid_iddata_timeline_SUM




In [15]:
accountId=get_accountId(summonerName, api_key)
summonerId=get_summonerId(summonerName, api_key)
TotalGamesCount=get_TotalGamesCount(summonerId, api_key)

# matchlist 만들기

In [16]:
start = time.time()  # 시작 시간 저장
 
 

matchlist=get_matchlist(accountId, Champion, Count, api_key)
print(len(matchlist))
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간


210
time : 0.9821524620056152


# ID랑 IDdata랑 Timeline 모으기

In [17]:

start = time.time()  # 시작 시간 저장
 
id_data_line_SUM=get_gameid_iddata_timeline_SUM(matchlist, api_key, api_key2, api_key3)
print(len(id_data_line_SUM))


print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간



changer :  0   matchIdData
iddata status code =  403 changer : 0
try 3 second wait time To 429error
iddata status code =  403 changer : 0
try 3 second wait time To 429error
changer :  0 matchTimeline
changer :  1   matchIdData
changer :  1 matchTimeline
changer :  2   matchIdData
changer :  2 matchTimeline
changer :  0   matchIdData
changer :  0 matchTimeline
changer :  1   matchIdData
changer :  1 matchTimeline
changer :  2   matchIdData
changer :  2 matchTimeline
changer :  0   matchIdData
changer :  0 matchTimeline
changer :  1   matchIdData
changer :  1 matchTimeline
changer :  2   matchIdData
changer :  2 matchTimeline
changer :  0   matchIdData
changer :  0 matchTimeline
changer :  1   matchIdData
changer :  1 matchTimeline
changer :  2   matchIdData
changer :  2 matchTimeline
changer :  0   matchIdData
changer :  1   matchIdData
changer :  1 matchTimeline
changer :  2   matchIdData
changer :  2 matchTimeline
changer :  0   matchIdData
changer :  0 matchTimeline
changer :  1   ma

changer :  2   matchIdData
changer :  2 matchTimeline
timeline_status_code =  429 changer :  2
try 3 second wait time To 429error
timeline_status_code =  429 changer :  2
try 3 second wait time To 429error
timeline_status_code =  429 changer :  2
try 3 second wait time To 429error
changer :  0   matchIdData
changer :  1   matchIdData
changer :  1 matchTimeline
changer :  2   matchIdData
changer :  2 matchTimeline
changer :  0   matchIdData
changer :  0 matchTimeline
changer :  1   matchIdData
changer :  1 matchTimeline
changer :  2   matchIdData
changer :  2 matchTimeline
changer :  0   matchIdData
changer :  0 matchTimeline
changer :  1   matchIdData
changer :  1 matchTimeline
changer :  2   matchIdData
changer :  2 matchTimeline
changer :  0   matchIdData
changer :  0 matchTimeline
changer :  1   matchIdData
changer :  1 matchTimeline
changer :  2   matchIdData
changer :  2 matchTimeline
changer :  0   matchIdData
changer :  1   matchIdData
changer :  1 matchTimeline
changer :  2   m

In [ ]:
import pickle

with open(summonerName+"_"+Champion+"_"+str(Count)+".pickle", 'wb') as f:
    pickle.dump(id_data_line_SUM, f, pickle.HIGHEST_PROTOCOL)
with open(summonerName+"_"+Champion+"_"+str(Count)+".pickle", 'rb') as f:
    data = pickle.load(f)

# 포지션 나누기

In [ ]:

def posion_define(data):
    IDdata=data[1]
    Timeline=data[2]
    top=[]
    jug=[]
    mid=[]
    bot=[]
    sup=[]
    err=[]

    for idnum in range(1,11):
        endminute=len(Timeline['frames'])-1
        x_posi=[]
        y_posi=[]
        partId=Timeline['frames'][0]['participantFrames'][str(idnum)]['participantId']    
        if (IDdata['participants'][partId-1]['spell1Id']==11)or( IDdata['participants'][partId-1]['spell2Id']==11):
            #print('participantId ',partId, '의 포지션 : Jungle  챔피언 : ',ckt(IDdata['participants'][partId-1]['championId']))
            jug.append(ckt(IDdata['participants'][partId-1]['championId']))


        else:
            for minute in range(0,  8):
                position=Timeline['frames'][minute]['participantFrames'][str(idnum)]['position']
                partId=Timeline['frames'][minute]['participantFrames'][str(idnum)]['participantId']
                x_posi.append( round ((position['x'] *  (512/15000)  ),2))
                y_posi.append(512- round ((position['y'] *  (512/15000)  ),2))

            else:
                x = x_posi
                y = y_posi
                #max_lane, lane = LanePredict(cur_player, support_index==str(j), jungle_index)
                #print(x)
                #print(y)
            if Timeline['frames'][7]['participantFrames'][str(idnum)]['minionsKilled']<25:
                #print(Timeline['frames'][6]['participantFrames'][str(idnum)]['participantId'], '는 서폿, 챔피언은 : ', ckt(IDdata['participants'][partId-1]['championId']))
                sup.append(ckt(IDdata['participants'][partId-1]['championId']))

            else:
                #print('participantId ',partId, '의 포지션 : ',LanePredict(x_posi, y_posi)[0], '  챔피언 : ', ckt(IDdata['participants'][partId-1]['championId']))
                if LanePredict(x_posi, y_posi)[0]=='TOP':
                    top.append(ckt(IDdata['participants'][partId-1]['championId']))
                elif LanePredict(x_posi, y_posi)[0]=='MID':
                    mid.append(ckt(IDdata['participants'][partId-1]['championId']))
                elif LanePredict(x_posi, y_posi)[0]=='BOT_CARRY':
                    bot.append(ckt(IDdata['participants'][partId-1]['championId']))
                else:
                    err.append(ckt(IDdata['participants'][partId-1]['championId']))
    if (len(top)==len(jug)==len(mid)==len(bot)==len(sup))==0:
        print(top,jug,mid,bot,sup)                

In [ ]:
for a in range(0,50):
    data=id_data_line_SUM[a]
    posion_define(data)
    print('\n')

In [ ]:
x1 = np.random.uniform(20, 60, 1000)
y1 = np.random.uniform(30, 220, 1000)

x2 = np.random.uniform(20, 150, 1500)
y2 = np.random.uniform(20, 160, 1500)

x3 = np.random.uniform(30, 200, 1000)
y3 = np.random.uniform(20, 60, 1000)
maplink = 'https://s3-us-west-1.amazonaws.com/riot-developer-portal/docs/map11.png'
image = io.imread(maplink)
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.scatter(x1, y1, c='black', s=50)
ax.scatter(x2, y2, c='black', s=50)
ax.scatter(x3, y3, c='black', s=50)
ax.axis('off')
ax.imshow(image)
plt.show()

In [ ]:
def SupJugPredict(data, player_spells, i): # 서폿과 정글을 판단하기 위한 함수
    if i==1: # 블루팀
        final_timeline = data.iloc[-10:-5].copy()
        spells = player_spells[:5]
    if i==6: # 레드팀
        final_timeline = data.iloc[-5:].copy()
        spells = player_spells[5:10]
    smite_count = (spells == 11).sum()
    # 각 팀의 스마이트 개수 (스마이트가 2명 이상이 보유하면, 라인의 위치로 판단하기 위함)
    if smite_count == 0: jungle_index = final_timeline['jungleMinionsKilled'].idxmax()
    elif  smite_count > 1: jungle_index = False
    else: jungle_index = np.where(spells == 11)[0][0] + i # 강타를 들고있는 플레이어 번호

    final_timeline = final_timeline.drop(index=final_timeline['jungleMinionsKilled'].idxmax()) # 정글 제외
    return final_timeline['minionsKilled'].idxmin(), str(jungle_index) # 정글을 제외하고 cs가 가장 적은 사람이 서포터

def LanePredict(x_posi, y_posi, support_bool=False, jungle_bool=True):
    lane = {'TOP': 0, 'MID': 0, 'BOT_CARRY': 0, 'JUNGLE': 0}
    etc = 0
    for xi, yi in zip(x_posi, y_posi):
        if (xi > 20) & (xi < 60) & (yi > 30) & (yi < 220): lane['TOP'] += 1
        elif (xi > 20) & (xi < 150) & (yi > 20) & (yi < 160): lane['TOP'] += 1
        elif (xi > 30) & (xi < 200) & (yi > 20) & (yi < 60): lane['TOP'] += 1
        elif (xi > 195) & (xi < 265) & (yi > 250) & (yi < 310): lane['MID'] += 1
        elif (xi > 220) & (xi < 295) & (yi > 220) & (yi < 290): lane['MID'] += 1
        elif (xi > 250) & (xi < 320) & (yi > 200) & (yi < 260): lane['MID'] += 1
        elif (xi > 290) & (xi < 350) & (yi > 160) & (yi < 215): lane['MID'] += 1
        elif (xi > 160) & (xi < 220) & (yi > 290) & (yi < 340): lane['MID'] += 1
        elif (xi > 310) & (xi < 460) & (yi > 435) & (yi < 485): lane['BOT_CARRY'] += 1
        elif (xi > 400) & (xi < 490) & (yi > 385) & (yi < 480): lane['BOT_CARRY'] += 1
        elif (xi > 440) & (xi < 490) & (yi > 310) & (yi < 455): lane['BOT_CARRY'] += 1
        elif (xi > 0) & (xi < 170) & (yi > 340) & (yi < 512): etc += 1
        elif (xi > 340) & (xi < 512) & (yi > 0) & (yi < 170): etc += 1
        else: lane['JUNGLE'] += 1
    if jungle_bool:
        del lane['JUNGLE']
    if support_bool & (max(lane, key=lane.get) == 'BOT_CARRY'): return 'BOT_SUPPORT', lane
    return max(lane, key=lane.get), lane

In [ ]:
smPartId

In [ ]:
smName

In [ ]:
championKey

In [ ]:
df[1][1]['participants'][summoner_participantId-1]

In [ ]:
MapScaler({'x':3400.0, 'y':5700.0})

In [ ]:
spell_api='http://ddragon.leagueoflegends.com/cdn/9.3.1/data/ko_KR/summoner.json'
R_spell=requests.get(spell_api)
spell_df=pd.DataFrame(R_spell.json())
spell_df

In [ ]:
x_posi

# 분당 위치 찍기

In [ ]:
def drawing_position(x,y):
    maplink = 'https://s3-us-west-1.amazonaws.com/riot-developer-portal/docs/map11.png'
    image = io.imread(maplink)
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    ax.scatter(x, y, c='red', s=90)
    ax.axis('off')
    ax.imshow(image)
    plt.show()

# MongoDB 적용

In [ ]:
import pymongo

conn = pymongo.MongoClient('mongodb://13.209.145.101')
loltest = conn.loltest
test=loltest.loltest

start = time.time()  # 시작 시간 저장
 


for i in range(0,len(matchlist)):
    dicDB = {}
    matchId=matchlist[i]
    dicDB['gameId'] = matchId
    dicDB['IDdata'] = get_matchIdData(matchId,api_key)
    dicDB['TLdata'] = get_matchtimeline(matchId,api_key)
    test.insert_one(dicDB)

    
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간